Notebook prepared by Henrique Lopes Cardoso (hlc@fe.up.pt), based on [Word2Vec Tutorial Notebook](https://github.com/kavgan/nlp-in-practice/tree/master/word2vec) by Kavita Ganesan and on [Gensim's documentation on the Word2Vec Model](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html).

# WORD EMBEDDINGS

## Word2Vec in Gensim

[Word2Vec](https://code.google.com/archive/p/word2vec/) is a model for training word embeddings that revolutionized the way words are represented. [Gensim](https://radimrehurek.com/gensim_3.8.3/models/word2vec.html) provides an implementation of the algorithm, with which we can train our own word embeddings.

#### The data

Training embeddings requires a big corpus, the bigger the better.

For illustration purposes, we'll make use of the (not very big) [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset, which includes full reviews of cars and hotels. More specifically, we'll use an 84MB compressed file with 255404 hotel reviews.

This is how each review looks like:

In [1]:
import gzip

data_file="reviews_data.txt.gz"

with gzip.open ('data/reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break

b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

Let's read the whole dataset into a list, while providing some logging information.

In the process of reading the data directly from the compressed file, we'll perform some pre-processing of the reviews using [gensim.utils.simple_preprocess](https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html). This does some basic pre-processing such as tokenization and lowercasing, and returns back a list of tokens (words).

In [2]:
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess(line)
    logging.info("Done reading data file")

In [4]:
# compressed file with the data
data_file="data/reviews_data.txt.gz"

# read the tokenized reviews into a list
documents = list(read_input(data_file))

len(documents)

2023-06-01 16:07:10,643 : INFO : reading file data/reviews_data.txt.gz...this may take a while
2023-06-01 16:07:10,645 : INFO : read 0 reviews
2023-06-01 16:07:12,516 : INFO : read 10000 reviews
2023-06-01 16:07:14,381 : INFO : read 20000 reviews
2023-06-01 16:07:16,505 : INFO : read 30000 reviews
2023-06-01 16:07:18,533 : INFO : read 40000 reviews
2023-06-01 16:07:20,905 : INFO : read 50000 reviews
2023-06-01 16:07:23,035 : INFO : read 60000 reviews
2023-06-01 16:07:24,870 : INFO : read 70000 reviews
2023-06-01 16:07:26,527 : INFO : read 80000 reviews
2023-06-01 16:07:28,292 : INFO : read 90000 reviews
2023-06-01 16:07:29,976 : INFO : read 100000 reviews
2023-06-01 16:07:31,667 : INFO : read 110000 reviews
2023-06-01 16:07:33,407 : INFO : read 120000 reviews
2023-06-01 16:07:35,161 : INFO : read 130000 reviews
2023-06-01 16:07:38,208 : INFO : read 140000 reviews
2023-06-01 16:07:39,865 : INFO : read 150000 reviews
2023-06-01 16:07:41,617 : INFO : read 160000 reviews
2023-06-01 16:07:4

255404

Each review item becomes a list of words, so what we have is a list of lists.

In [5]:
print(documents[0])

['oct', 'nice', 'trendy', 'hotel', 'location', 'not', 'too', 'bad', 'stayed', 'in', 'this', 'hotel', 'for', 'one', 'night', 'as', 'this', 'is', 'fairly', 'new', 'place', 'some', 'of', 'the', 'taxi', 'drivers', 'did', 'not', 'know', 'where', 'it', 'was', 'and', 'or', 'did', 'not', 'want', 'to', 'drive', 'there', 'once', 'have', 'eventually', 'arrived', 'at', 'the', 'hotel', 'was', 'very', 'pleasantly', 'surprised', 'with', 'the', 'decor', 'of', 'the', 'lobby', 'ground', 'floor', 'area', 'it', 'was', 'very', 'stylish', 'and', 'modern', 'found', 'the', 'reception', 'staff', 'geeting', 'me', 'with', 'aloha', 'bit', 'out', 'of', 'place', 'but', 'guess', 'they', 'are', 'briefed', 'to', 'say', 'that', 'to', 'keep', 'up', 'the', 'coroporate', 'image', 'as', 'have', 'starwood', 'preferred', 'guest', 'member', 'was', 'given', 'small', 'gift', 'upon', 'check', 'in', 'it', 'was', 'only', 'couple', 'of', 'fridge', 'magnets', 'in', 'gift', 'box', 'but', 'nevertheless', 'nice', 'gesture', 'my', 'room

#### Training the Word2Vec model

To train a Word2Vec model, we instantiate Word2Vec and pass it the text we have loaded before. You can check the available options for [instantiation](https://radimrehurek.com/gensim_3.8.3/models/word2vec.html#gensim.models.word2vec.Word2Vec) and for [training](https://radimrehurek.com/gensim_3.8.3/models/word2vec.html#gensim.models.word2vec.Word2Vec.train).

Training a Word2Vec model takes time, depending on your hardware. In this particular case, expect training to take something between 5 to 10 minutes on an Intel Core-i7 16GB desktop. I know, that's a very wide range, but of course it depends on which other processes are running in the same machine...

In [6]:
from datetime import datetime

start_time = datetime.now()

model = gensim.models.Word2Vec(documents, vector_size=150, window=10, min_count=2, workers=10, sg=1)

print("Training time:", datetime.now() - start_time)

2023-06-01 16:08:20,186 : INFO : collecting all words and their counts
2023-06-01 16:08:20,188 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-01 16:08:20,408 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2023-06-01 16:08:20,642 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2023-06-01 16:08:20,904 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2023-06-01 16:08:21,147 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2023-06-01 16:08:21,412 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2023-06-01 16:08:21,674 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2023-06-01 16:08:21,892 : INFO : PROGRESS: at sentence #70000, processed 12637528 words, keeping 83199 word types
2023-06-01 16:08:22,098 : INFO : PROG

2023-06-01 16:08:53,502 : INFO : EPOCH 0 - PROGRESS: at 35.02% examples, 432123 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:08:54,523 : INFO : EPOCH 0 - PROGRESS: at 36.46% examples, 430485 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:08:55,525 : INFO : EPOCH 0 - PROGRESS: at 37.88% examples, 429738 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:08:56,527 : INFO : EPOCH 0 - PROGRESS: at 38.79% examples, 423652 words/s, in_qsize 18, out_qsize 1
2023-06-01 16:08:57,539 : INFO : EPOCH 0 - PROGRESS: at 39.70% examples, 418325 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:08:58,541 : INFO : EPOCH 0 - PROGRESS: at 41.10% examples, 416925 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:08:59,599 : INFO : EPOCH 0 - PROGRESS: at 42.57% examples, 415973 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:09:00,626 : INFO : EPOCH 0 - PROGRESS: at 44.18% examples, 416168 words/s, in_qsize 20, out_qsize 0
2023-06-01 16:09:01,668 : INFO : EPOCH 0 - PROGRESS: at 45.60% examples, 414661 words/s,

2023-06-01 16:10:06,255 : INFO : EPOCH 1 - PROGRESS: at 32.64% examples, 420367 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:07,256 : INFO : EPOCH 1 - PROGRESS: at 34.07% examples, 420869 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:08,274 : INFO : EPOCH 1 - PROGRESS: at 35.59% examples, 421084 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:09,280 : INFO : EPOCH 1 - PROGRESS: at 37.10% examples, 421393 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:10,281 : INFO : EPOCH 1 - PROGRESS: at 38.65% examples, 421463 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:11,284 : INFO : EPOCH 1 - PROGRESS: at 40.09% examples, 421084 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:12,295 : INFO : EPOCH 1 - PROGRESS: at 41.76% examples, 421547 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:13,297 : INFO : EPOCH 1 - PROGRESS: at 43.26% examples, 421817 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:10:14,308 : INFO : EPOCH 1 - PROGRESS: at 44.84% examples, 421587 words/s,

2023-06-01 16:11:19,452 : INFO : EPOCH 2 - PROGRESS: at 25.55% examples, 406119 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:20,462 : INFO : EPOCH 2 - PROGRESS: at 27.11% examples, 406562 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:21,479 : INFO : EPOCH 2 - PROGRESS: at 28.66% examples, 407081 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:22,481 : INFO : EPOCH 2 - PROGRESS: at 30.12% examples, 407481 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:23,507 : INFO : EPOCH 2 - PROGRESS: at 31.76% examples, 407800 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:24,529 : INFO : EPOCH 2 - PROGRESS: at 33.25% examples, 408353 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:25,552 : INFO : EPOCH 2 - PROGRESS: at 34.69% examples, 408967 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:26,563 : INFO : EPOCH 2 - PROGRESS: at 36.22% examples, 409635 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:11:27,579 : INFO : EPOCH 2 - PROGRESS: at 37.70% examples, 410177 words/s,

2023-06-01 16:12:32,290 : INFO : EPOCH 3 - PROGRESS: at 25.98% examples, 412832 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:33,334 : INFO : EPOCH 3 - PROGRESS: at 27.66% examples, 413241 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:34,351 : INFO : EPOCH 3 - PROGRESS: at 29.23% examples, 414055 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:35,369 : INFO : EPOCH 3 - PROGRESS: at 30.73% examples, 414230 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:36,393 : INFO : EPOCH 3 - PROGRESS: at 32.34% examples, 414563 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:37,392 : INFO : EPOCH 3 - PROGRESS: at 33.78% examples, 415238 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:38,428 : INFO : EPOCH 3 - PROGRESS: at 35.27% examples, 415370 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:39,467 : INFO : EPOCH 3 - PROGRESS: at 36.84% examples, 415669 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:12:40,474 : INFO : EPOCH 3 - PROGRESS: at 38.38% examples, 416088 words/s,

2023-06-01 16:13:44,921 : INFO : EPOCH 4 - PROGRESS: at 20.28% examples, 379427 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:45,940 : INFO : EPOCH 4 - PROGRESS: at 21.59% examples, 379778 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:46,994 : INFO : EPOCH 4 - PROGRESS: at 22.62% examples, 379580 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:48,024 : INFO : EPOCH 4 - PROGRESS: at 23.68% examples, 380472 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:49,043 : INFO : EPOCH 4 - PROGRESS: at 24.67% examples, 380139 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:50,063 : INFO : EPOCH 4 - PROGRESS: at 26.03% examples, 379303 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:51,069 : INFO : EPOCH 4 - PROGRESS: at 27.45% examples, 379082 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:52,087 : INFO : EPOCH 4 - PROGRESS: at 28.94% examples, 379808 words/s, in_qsize 19, out_qsize 0
2023-06-01 16:13:53,100 : INFO : EPOCH 4 - PROGRESS: at 30.18% examples, 379141 words/s,

Training time: 0:06:23.686567


#### Exploiting the Word2Vec model

We can now inspect the word embeddings that we have trained. We can start by looking at the embeddings of a specific word:

In [7]:
# embeddings
model.wv.get_vector("dirty")

array([-3.56595814e-01,  8.00722480e-01, -1.50343150e-01,  3.13106596e-01,
       -1.78973153e-02, -5.66125056e-03,  3.82428877e-02, -1.42356619e-01,
        4.25188988e-02, -1.57804161e-01,  1.17257513e-01,  6.63125589e-02,
        1.26103126e-02,  2.60465324e-01, -6.76033273e-02,  1.01392619e-01,
       -1.00547954e-01,  2.61664748e-01, -5.45448512e-02, -3.01816851e-01,
        3.32458653e-02,  1.47133857e-01,  1.36648372e-01, -2.31912985e-01,
       -3.82015333e-02, -7.76402727e-02, -8.51919800e-02, -2.91402754e-03,
        1.42763302e-01,  3.01458448e-01, -1.55419648e-01,  9.55241621e-02,
       -1.12980437e-02, -5.54107837e-02, -1.55068651e-01, -4.73339632e-02,
        1.02313414e-01,  3.23405638e-02, -1.18305452e-01, -5.28339595e-02,
        3.72817904e-01, -7.04257339e-02, -8.35831277e-03, -1.83132619e-01,
       -2.48425186e-01,  4.73089516e-03, -1.88391097e-02, -2.93081850e-01,
        2.75182277e-01, -1.06561571e-01, -3.70598704e-01,  5.49557447e-01,
       -1.16988003e-01,  

Which are the words most similar to this one?

In [8]:
# similarity
w1 = "dirty"
model.wv.most_similar(positive=w1)

[('filthy', 0.8905879855155945),
 ('smelly', 0.8316246867179871),
 ('unclean', 0.8263578414916992),
 ('dusty', 0.8183785080909729),
 ('dingy', 0.8093951344490051),
 ('stained', 0.8060740828514099),
 ('grubby', 0.7994816899299622),
 ('moldy', 0.7968888282775879),
 ('disgusting', 0.7818449139595032),
 ('grimy', 0.7814695835113525)]

You can also limit to a smaller number of hits:

In [9]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar(positive=w1, topn=6)

[('courteous', 0.9033957123756409),
 ('professional', 0.8547850251197815),
 ('attentive', 0.8318154215812683),
 ('curtious', 0.8219287395477295),
 ('efficient', 0.8191940188407898),
 ('obliging', 0.8000189065933228)]

In [10]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar(positive=w1, topn=6)

[('germany', 0.6906672120094299),
 ('england', 0.6704092025756836),
 ('spain', 0.6393370628356934),
 ('barcelona', 0.6390076875686646),
 ('paris', 0.6381335854530334),
 ('malaysia', 0.6348099112510681)]

We can also provide to *most_similar* not only positive concepts, but also negative ones. This allows us to do some arithmetic on the vector representations for certain sets of words!

The famous example **king - man + woman = queen** goes as follows:

In [11]:
# arithmetic: vec(“king”) - vec(“man”) + vec(“woman”) =~ vec(“queen”)
w1 = ["king",'woman']
w2 = ['man']
model.wv.most_similar(positive=w1, negative=w2, topn=1)

[('queen', 0.7838140726089478)]

We can get the similarity scores for specific word pairs.

In [12]:
# similarity between two different words
model.wv.similarity(w1="dirty", w2="smelly")

0.8316246

In [13]:
# similarity between two identical words
model.wv.similarity(w1="dirty", w2="dirty")

1.0

In [14]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty", w2="clean")

0.3736205

And we can check which word in a list of words is an intruder:

In [15]:
model.wv.doesnt_match(["cat", "dog", "france"])

'france'

In [16]:
model.wv.doesnt_match(["bed", "pillow", "duvet", "shower"])

'shower'

In [17]:
model.wv.doesnt_match(["car", "bicycle", "plane", "skate"])

'plane'

In [18]:
model.wv.doesnt_match(["car", "bicycle", "bus", "trolley"])

'bicycle'

Make your own experiments! Try to find out:
- Which word is most similar to *lift*?
- What are the 3 words most similar to *crab*?
- How similar are the words *waitress* and *waiter*?
- If you take *portugal*, remove *lisbon*, and add *dublin*, what do you get?

In [ ]:
# your code here


#### Saving and loading a Word2Vec model

You can save a trained model so that you are able to load it again in the future, and optionally continue training it.

In [19]:
# save full model (including trainable vectors to resume training)
model.save("reviews_model")

2023-06-01 16:19:34,644 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'reviews_model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-06-01T16:19:34.644326', 'gensim': '4.2.0', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2023-06-01 16:19:34,661 : INFO : storing np array 'vectors' to reviews_model.wv.vectors.npy
2023-06-01 16:19:34,741 : INFO : storing np array 'syn1neg' to reviews_model.syn1neg.npy
2023-06-01 16:19:34,810 : INFO : not storing attribute cum_table
2023-06-01 16:19:34,870 : INFO : saved reviews_model


In [20]:
# load full model
model = gensim.models.Word2Vec.load("reviews_model")

2023-06-01 16:20:01,839 : INFO : loading Word2Vec object from reviews_model
2023-06-01 16:20:01,931 : INFO : loading wv recursively from reviews_model.wv.* with mmap=None
2023-06-01 16:20:01,932 : INFO : loading vectors from reviews_model.wv.vectors.npy with mmap=None
2023-06-01 16:20:01,966 : INFO : loading syn1neg from reviews_model.syn1neg.npy with mmap=None
2023-06-01 16:20:02,004 : INFO : setting ignored attribute cum_table to None
2023-06-01 16:20:02,567 : INFO : Word2Vec lifecycle event {'fname': 'reviews_model', 'datetime': '2023-06-01T16:20:02.567180', 'gensim': '4.2.0', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


#### Saving and loading the word embeddings

If you're sure you won't be training the model any longer, you can save its *KeyedVectors* (the word embeddings).

In [21]:
# save model word vectors
model.wv.save("reviews_wv")

2023-06-01 16:20:09,731 : INFO : KeyedVectors lifecycle event {'fname_or_handle': 'reviews_wv', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-06-01T16:20:09.731879', 'gensim': '4.2.0', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2023-06-01 16:20:09,732 : INFO : storing np array 'vectors' to reviews_wv.vectors.npy
2023-06-01 16:20:09,788 : INFO : saved reviews_wv


After saving the embeddings, you can load them and use them.

In [22]:
# load model word vectors
wv = gensim.models.KeyedVectors.load("reviews_wv")

print(wv.most_similar(positive="lift", topn=1))

2023-06-01 16:20:29,967 : INFO : loading KeyedVectors object from reviews_wv
2023-06-01 16:20:30,008 : INFO : loading vectors from reviews_wv.vectors.npy with mmap=None
2023-06-01 16:20:30,035 : INFO : KeyedVectors lifecycle event {'fname': 'reviews_wv', 'datetime': '2023-06-01T16:20:30.035488', 'gensim': '4.2.0', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


[('lifts', 0.8314789533615112)]


## Visualization

Word embeddings can be visualized by reducing dimensionality of the words to 2 dimensions using [tSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html).

Given enough training data, we can observe certain patterns in the vector space, including:
- Semantic relations: words like *cat*, *dog*, *cow*, etc. have a tendency to lie close by.
- Syntactic relations: words like *run*, *running* or *cut*, *cutting* lie close together.
- Arithmetic properties such as *King - Man = Queen - Woman*.

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import random


def reduce_dimensions(model, num_dimensions=2, words=[]):

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    word_count = 0
    
    # if no word list is given, assume we want to use the whole data in the model
    if(words == []):
        words = model.wv.index_to_key

    for word in words:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels


# 2 dimension plotting
def plot_with_matplotlib(x_vals, y_vals, labels, words=[]):

    random.seed(0)
    
    x_vals_new = np.array([])
    y_vals_new = np.array([])
    labels_new = np.array([])
    if(words == []):
        # if no word list is given, assume we want to plot the whole data
        x_vals_new = x_vals
        y_vals_new = y_vals
        labels_new = labels
    else:
        for i in range(len(labels)):
            if(labels[i] in words):
                x_vals_new = np.append(x_vals_new,x_vals[i])
                y_vals_new = np.append(y_vals_new,y_vals[i])
                labels_new = np.append(labels_new,labels[i])
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals_new, y_vals_new)

    # apply labels
    for i in range(len(labels_new)):
        plt.annotate(labels_new[i], (x_vals_new[i], y_vals_new[i]))
    
    plt.show()

In [ ]:
words = []
words.extend(["king", "man", "queen", "woman"])

vectors, labels = reduce_dimensions(model, 2, words)
x_vals = [v[0] for v in vectors]
y_vals = [v[1] for v in vectors]
print(x_vals, y_vals, labels)

In [ ]:
plot_with_matplotlib(x_vals, y_vals, labels, ["king", "man", "queen", "woman"])

## Portuguese embeddings

A number of embeddings for Portuguese are available at [NILC](http://nilc.icmc.usp.br/embeddings), as well as at the [NLX-group](https://github.com/nlx-group/LX-DSemVectors).

In [ ]:
from gensim.models import KeyedVectors

# takes a while to load...
model_pt = KeyedVectors.load_word2vec_format('skip_s100.txt')

In [ ]:
# save model word vectors
model_pt.save("pt_wv")

In [ ]:
# load model word vectors (much faster than the above)
model_pt = gensim.models.KeyedVectors.load("pt_wv")

In [ ]:
model_pt.most_similar(positive=["cão"])

In [ ]:
model_pt.most_similar(positive=["rei", "mulher"], negative=["homem"])

Make your own experiments!

In [ ]:
# your code here
